In [3]:
#libraries
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import sys
from IPython.display import clear_output
import time
from fake_useragent import UserAgent
import random as rd
import time

#an open source progress bar, cuz why not?
def progressBar(count_value, total, suffix=''):
    bar_length = 100
    filled_up_Length = int(round(bar_length* count_value / float(total)))
    percentage = round(100.0 * count_value/float(total),1)
    bar = '=' * filled_up_Length + '-' * (bar_length - filled_up_Length)
    sys.stdout.write('[%s] %s%s ...%s\r' %(bar, percentage, '%', suffix))
    sys.stdout.flush()


class scraper():
    def __init__(self, timer=10,testing=False, hidden=False):
        '''An Innit Innit thing, am i right init!?'''
        self.web="https://www.phonearena.com/phones/page/"
        self.timer = timer
        tab1=['Size', 'Resolution', 'Technology', 'Refresh rate', 'Features', 'System chip', 'Processor', 'GPU', 'RAM', 'Internal storage', 'Device type', 'OS', 'Capacity', 'Charging', 'Max charge speed', 'Rear', 'Main camera', 'Second camera', 'Third camera', 'Video recording', 'Front', 'Resistance', 'Biometrics', 'Keys', '5G', 'Data Speed', 'SIM type', 'Headphones', 'Speakers']
        tab2=['Name', 'Release date']
        self.dataspec=pd.DataFrame(columns=tab1)
        self.dataname=pd.DataFrame(columns=tab2)
        self.testing=testing
        self.hidden=1 if hidden else 0
        
    #Spoofing
    def getidentity(self, identity=10, proxies=20, debug=False):
        self.debug=1 if debug else 0
        '''Get a random users and proxies. Why should i be respectful? Cuh.'''
        self.useragentlist=[UserAgent().random for i in range(identity)]
        proxydata=pd.DataFrame(columns=["IP", "Port", "Country", "Anonymity", "Last Checked"])
        header= {'User-Agent': str(rd.sample(self.useragentlist, 1)), "Accept-Language": "en-US, en;q=0.5"}
        SSLPROXIES=requests.get("https://www.sslproxies.org/", headers= header)
        soup=BeautifulSoup(SSLPROXIES.content, "html5lib")
        table=soup.find("table", attrs={"class" : "table table-striped table-bordered"}).find("tbody")
        counter=0
        for i in table.find_all("tr"):
                if counter==proxies:
                        break
                values=[j.text for j in i.find_all("td")]
                proxydata.loc[len(proxydata)]=[values[0], values[1], values[3], values[4], values[7]]
                counter+=1
        self.proxydata=proxydata
        self.brute()
        
    def brute(self):
        '''BruteChecker'''
        usableproxyindex=[]                                                              #index
        for i in range(len(self.proxydata)):
            rowproxydata=self.proxydata.iloc[i,:].values
            proxy= {
            'http': f'http://{rowproxydata[0]}:{rowproxydata[1]}',
            'https': f'http://{rowproxydata[0]}:{rowproxydata[1]}',
            }
            if self.debug:
                print(f'Trying to connect {rowproxydata[0]}:{rowproxydata[1]} |{rowproxydata[2]}|', end="status: ")
            try:
                r=requests.get("https://www.phonearena.com/", proxies=proxy, timeout=15)
                if  BeautifulSoup(r.content, 'html5lib').find("title").text != "Too Many Requests":
                    usableproxyindex.append(i)
                    print("Success")
                else:
                    print("failed")
            except:
                print("failed")
                pass
        self.proxydata=self.proxydata.iloc[usableproxyindex,:]
        
    #getting the phone spec
    def getphonespec(self, r):
        phonespectable=BeautifulSoup(r.content, "html5lib")

        name=phonespectable.find("header").text[1:-1].replace(" Specs", "")
        releasedate=phonespectable.find("p", attrs={"class": "widgetQuickSpecs__title_paragraph"}).text
        self.dataname.loc[len(self.dataname)]=[name, releasedate]

        moretable=phonespectable.find("div", attrs={"class": "widgetSpecs"})
        tempdict={}
        for j in moretable.find_all("tbody"):
            for k in j.find_all("tr"):
                attr, *val=k.text[1:-2].split("\n")
                attr=attr.strip(":")
                tempdict[attr]=val
        new_row = {}
        for col_name in self.dataspec.columns:
            if col_name in tempdict:
                new_row[col_name] = tempdict[col_name]
            else:
                new_row[col_name] = "No information"  # Or some default value
        self.dataspec = self.dataspec.append(new_row, ignore_index=True)
        clear_output()

    #function for getting a request
    def requestget(self, link):
        if not self.hidden:
            time.sleep(self.timer)
            return requests.get(link)
        while True:
            try:
                rduseragent=str(rd.sample(self.useragentlist, 1))
                header= {'User-Agent': rduseragent, "Accept-Language": "en-US, en;q=0.5"}
                rdrproxydata=self.proxydata.iloc[rd.randint(0,len(self.proxydata)-1),:].values
                if self.debug:
                    print(f'Connected to {rdrproxydata[0]}:{rdrproxydata[1]} | {rdrproxydata[2]}    | {rduseragent}')
                proxy= {
                    'http': f'http://{rdrproxydata[0]}:{rdrproxydata[1]}',
                    'https': f'http://{rdrproxydata[0]}:{rdrproxydata[1]}',
                    }
                time.sleep(self.timer)
                asu=requests.get(link, headers=header, proxies=proxy)
                return asu
            except:
                if self.debug:
                    print("retrying")
                pass
    
    #Start the scrape!
    def startscrape(self,startpage, endpage):
        counter=0
        for j in range(startpage, endpage+1):
            r=self.requestget(self.web+str(j))
            soup=BeautifulSoup(r.content, "html5lib").find("div", attrs={"class" : "stream block"})
            counter2=1
            for i in soup.find_all("a", attrs={"class" : "thumbnail"}):
                print(f'{i["href"]}\nPage: {j}')
                print(f'{counter2}/{len(soup.find_all("a", attrs={"class" : "thumbnail"}))}')
                progressBar(counter, len(soup.find_all("a", attrs={"class" : "thumbnail"}))*(endpage-startpage+1))
                self.getphonespec(self.requestget(i["href"]))
                counter+=1
                counter2+=1
                if self.testing: break
            print("next page.")
    
    def data(self):
        return pd.concat((self.dataname,self.dataspec), axis=1)
    
    def save(self, path="No.csv"):
        yes=self.data()
        pd.DataFrame.to_csv(yes,path)

In [25]:
#without spoofing
test=scraper(1)
test.startscrape(1,1)               #scrape page 1
test.data()                         #show data

next page.


In [2]:
#with spoofing
test2=scraper(1, hidden=True)
test2.getidentity(debug=True)       #get 10 identities and 20 proxies [Default]
test2.startscrape(1,1)               #scrape page 1

Trying to connect 8.219.97.248:80 |Singapore|status: Success
Trying to connect 144.49.99.216:8080 |Singapore|status: failed
Trying to connect 20.44.206.138:80 |Singapore|status: Success
Trying to connect 20.204.190.254:3129 |India|status: Success
Trying to connect 3.16.22.118:3128 |United States|status: failed
Trying to connect 144.49.99.215:8080 |Singapore|status: failed
Trying to connect 222.109.192.34:8080 |South Korea|status: Success
Trying to connect 144.49.99.17:8080 |Singapore|status: failed
Trying to connect 34.87.130.22:8080 |Singapore|status: failed
Trying to connect 20.219.177.85:3129 |India|status: Success
Trying to connect 20.219.177.73:3129 |India|status: failed
Trying to connect 20.204.212.45:3129 |India|status: failed
Trying to connect 20.219.178.121:3129 |India|status: failed
Trying to connect 20.219.180.149:3129 |India|status: failed
Trying to connect 20.204.214.79:3129 |India|status: failed
Trying to connect 186.121.235.66:8080 |Bolivia|status: failed
Trying to conne

Testing thingy=====================================================================

In [ ]:
test.save("Yespapa.csv")            #Save scrapped into csv

In [11]:
r=requests.get("https://www.phonearena.com/phones/Motorola-Moto-G53_id12106")

In [12]:
soup=BeautifulSoup(r.content, "html5lib").find("div", attrs={"class" : "layout__page_main"})

In [13]:
print(soup.prettify())

<div class="layout__page_main">
 <script id="fullscreen-gallery-template" type="text/html">
  <div class="slide-show-gallery" data-device-orientation="landscape" style="display:none">
        <div class="slide-show-gallery-controls">
            <a href="javascript:;" class="gallery-list-view" title="View All">
                <i class="zmdi zmdi-apps"></i>
            </a>
            <a href="javascript:;" class="slide-show-gallery-close" title="Exit">
                <i class="zmdi zmdi-close"></i>
            </a>
        </div>
        <div class="slide-show-gallery-container">
            <div class="preview-slider">
                <a href="javascript:;" class="image-nav image-nav-left"
                   title="use left arrow key &#8592 &#13to navigate to the previous image">
                    <i class="zmdi zmdi-chevron-left"></i>
                </a>
                <div class="preview-slider-images">
                    <div class="" data-id="0" data-width="0" data-height=

In [14]:
import pandas as pd

In [34]:
#releasedate
tab1=['Size', 'Resolution', 'Technology', 'Refresh rate', 'Features', 'System chip', 'Processor', 'GPU', 'RAM', 'Internal storage', 'Device type', 'OS', 'Capacity', 'Charging', 'Max charge speed', 'Rear', 'Main camera', 'Second camera', 'Third camera', 'Video recording', 'Front', 'Resistance', 'Biometrics', 'Keys', '5G', 'Data Speed', 'SIM type', 'Headphones', 'Speakers']
tab2=['Name', 'Release date']
thisdata=pd.DataFrame(columns=tab1)
thatdata=pd.DataFrame(columns=tab2)

name=soup.find("header").text[1:-1].replace(" Specs", "")
releasedate=soup.find("p", attrs={"class": "widgetQuickSpecs__title_paragraph"}).text
thatdata.loc[len(thatdata)]=[name, releasedate]

moretable=soup.find("div", attrs={"class": "widgetSpecs"})
tempdict={}
for j in moretable.find_all("tbody"):
    for k in j.find_all("tr"):
        attr, *val=k.text[1:-2].split("\n")
        attr=attr.strip(":")
        tempdict[attr]=val
new_row = {}
for col_name in thisdata.columns:
    if col_name in tempdict:
        new_row[col_name] = tempdict[col_name]
    else:
        new_row[col_name] = "No information"  # Or some default value
thisdata = thisdata.append(new_row, ignore_index=True)
thisdata

C:\Users\Rhey\AppData\Local\Temp\ipykernel_9808\2562945076.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thisdata = thisdata.append(new_row, ignore_index=True)


,Size,Resolution,Technology,Refresh rate,Features,System chip,Processor,GPU,RAM,Internal storage,...,Video recording,Front,Resistance,Biometrics,Keys,5G,Data Speed,SIM type,Headphones,Speakers
0,[6.5 inches],"[1600 x 720 pixels, 20:9 ratio, 270 PPI]",[IPS LCD],[120Hz],[Charging],[Qualcomm Snapdragon 480+ SM4350-AC (12 nm)],"[Octa-core, 2x 2.2Ghz + 6x 1.8GHz]",[Adreno 619],[4GB],[128GB],...,[Yes],[16 MP],No information,[Fingerprint (touch)],"[Right: Volume control, Lock/Unlock key]","[n1, n3, n5, n7, n8, n20, n28, n40, n41, n77, ...","[LTE-A, HSDPA+ (4G) 42.2 Mbit/s]",No information,[3.5mm jack],"[Earpiece, Multiple speakers]"
